In [1]:
import tensorflow as tf
import tensorflow_hub as hub

In [20]:
import matplotlib.pylab as plt
import os
from google.colab import files

In [3]:

# loading inception_resnet_v2 trained on imagenet dataset as per https://arxiv.org/pdf/1704.03549.pdf

# URL = "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1"
#feature_extractor = hub.KerasLayer(URL,
#                                   input_shape=(IMAGE_RES, IMAGE_RES,3))

main_model = tf.keras.applications.InceptionResNetV2(include_top=False, weights='imagenet', input_tensor=None, input_shape=None, pooling=None)


219062272/219055592 [==============================] - 2s 0us/step


In [4]:
# freezing the weights of the model and removing the last layer
main_model.trainable = False
main_model.layers.pop()

In [5]:
model = tf.keras.Sequential([
  main_model,
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_resnet_v2 (Functio (None, None, None, 1536)  54336736  
_________________________________________________________________
dense (Dense)                (None, None, None, 2)     3074      
Total params: 54,339,810
Trainable params: 3,074
Non-trainable params: 54,336,736
_________________________________________________________________


In [6]:
!git clone "https://github.com/oh-my-ocr/text_renderer"

Cloning into 'text_renderer'...
remote: Enumerating objects: 158, done.
remote: Counting objects: 100% (158/158), done.
remote: Compressing objects: 100% (96/96), done.
remote: Total 434 (delta 95), reused 105 (delta 59), pack-reused 276
Receiving objects: 100% (434/434), 11.57 MiB | 24.19 MiB/s, done.
Resolving deltas: 100% (170/170), done.


In [7]:
os.chdir("text_renderer")

In [8]:
!python3 setup.py develop

running develop
running egg_info
creating text_renderer.egg-info
writing text_renderer.egg-info/PKG-INFO
writing dependency_links to text_renderer.egg-info/dependency_links.txt
writing top-level names to text_renderer.egg-info/top_level.txt
writing manifest file 'text_renderer.egg-info/SOURCES.txt'
writing manifest file 'text_renderer.egg-info/SOURCES.txt'
running build_ext
Creating /usr/local/lib/python3.6/dist-packages/text-renderer.egg-link (link to .)
Adding text-renderer 0.0.1 to easy-install.pth file

Installed /content/text_renderer
Processing dependencies for text-renderer==0.0.1
Finished processing dependencies for text-renderer==0.0.1


In [9]:
!pip3 install -r docker/requirements.txt

     |████████████████████████████████| 25.4MB 163kB/s 
     |████████████████████████████████| 808kB 44.1MB/s 
     |████████████████████████████████| 2.1MB 35.3MB/s 
     |████████████████████████████████| 81kB 8.0MB/s 
     |████████████████████████████████| 61kB 6.4MB/s 
     |████████████████████████████████| 2.9MB 41.1MB/s 
     |████████████████████████████████| 10.8MB 45.0MB/s 
     |████████████████████████████████| 92kB 8.3MB/s 
     |████████████████████████████████| 102kB 9.5MB/s 
     |████████████████████████████████| 92kB 9.3MB/s 
     |████████████████████████████████| 122kB 39.1MB/s 
     |████████████████████████████████| 102kB 6.7MB/s 
  Created wheel for fire: filename=fire-0.3.1-py2.py3-none-any.whl size=111005 sha256=077005e10a70ce3bb3654a3590c452f7f66364ea2e851c05f0e5eba446567f9e
  Stored in directory: /root/.cache/pip/wheels/c1/61/df/768b03527bf006b546dce284eb4249b185669e65afc5fbb2ac
  Created wheel for contextvars: filename=contextvars-2.4-cp36-none-any.whl siz

In [10]:
!python3 main.py \
    --config example_data/example.py \
    --dataset img \
    --num_processes 2 \
    --log_period 10

2020-08-24 17:35:06.959 | INFO     | text_renderer.corpus.char_corpus:__init__:58 - load: /content/text_renderer/example_data/text/chn_text.txt
2020-08-24 17:35:06.960 | INFO     | text_renderer.corpus.char_corpus:__init__:58 - load: /content/text_renderer/example_data/text/eng_text.txt
2020-08-24 17:35:06.964 | INFO     | text_renderer.utils.utils:load_chars_file:172 - load 5071 chars from: /content/text_renderer/example_data/char/chn.txt
2020-08-24 17:35:06.964 | INFO     | text_renderer.corpus.corpus:filter_by_chars:122 - filtering text by chars...
2020-08-24 17:35:06.965 | INFO     | text_renderer.corpus.corpus:filter_by_chars:151 - Filter 10.14%(172) chars in input text。Unique chars(5): {'；', '：', '—', '\n', ' '}
2020-08-24 17:35:07.544 | INFO     | text_renderer.utils.utils:load_chars_file:172 - load 5071 chars from: /content/text_renderer/example_data/char/chn.txt
2020-08-24 17:35:07.544 | INFO     | text_renderer.corpus.corpus:filter_by_chars:122 - filtering text by chars...
20

In [27]:
!zip -r Dataset.zip /content/output

  adding: content/output/ (stored 0%)
  adding: content/output/rand_corpus/ (stored 0%)
  adding: content/output/rand_corpus/labels.json (deflated 69%)
  adding: content/output/rand_corpus/images/ (stored 0%)
  adding: content/output/rand_corpus/images/000000018.jpg (stored 0%)
  adding: content/output/rand_corpus/images/000000007.jpg (stored 0%)
  adding: content/output/rand_corpus/images/000000038.jpg (stored 0%)
  adding: content/output/rand_corpus/images/000000004.jpg (deflated 0%)
  adding: content/output/rand_corpus/images/000000013.jpg (stored 0%)
  adding: content/output/rand_corpus/images/000000003.jpg (stored 0%)
  adding: content/output/rand_corpus/images/000000044.jpg (deflated 0%)
  adding: content/output/rand_corpus/images/000000012.jpg (deflated 0%)
  adding: content/output/rand_corpus/images/000000047.jpg (stored 0%)
  adding: content/output/rand_corpus/images/000000031.jpg (stored 0%)
  adding: content/output/rand_corpus/images/000000028.jpg (stored 0%)
  adding: conte

In [23]:
files.download('Dataset.zip')

FileNotFoundError: ignored